In [1]:
from Data_Input import Input_Data 
from Data_Processing import Basic_FE
from Data_Splitting import train_test, train_test_val
import numpy as np
import pandas as pd
import importlib


#If things aren't working, restart kernel to force it to update imports
#because python is dumb!

In [2]:
# Inputs and processes data
data = Input_Data()
p_data = Basic_FE(data)
#processed_data.info()


In [4]:
#Splits and resamples data
split = train_test_val(p_data, 0.2,0.1)

2.000053812624442


In [7]:
#Fits and tests data

# Basic Idea
Make assumptions (feature engineering, what else..?)
Run models (test assumptions)

In [ ]:
# Do different ML systems in separate files
# Make a submission file as well
# Bootstrapping option
# Make ML model class to interface with rest of stuff




#Basic structure:

#X_test Y_test X_train_ Y_train

#import model1 from model1
#import model2 from model2
#
#models = [model1,model2,]
#
# for model in models:
# model1(processed_data) (prints data)